**Import Libraries**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import copy
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import matthews_corrcoef, confusion_matrix, accuracy_score
from scipy.stats import fisher_exact
from sklearn.preprocessing import StandardScaler

**Read In Data**

In [2]:
gene_df = pd.read_excel('/kaggle/input/pomeroy-et-al-brain-data/Dataset_C_MD_outcome2.gct (6).xlsx')

**Create a list of tuples formated ('patientIDs', 'patientStatus') where patientIDs is Brain_MD_X and patientStatus is 0 (dead) or 1 (alive)**

In [3]:
# Brin_MD_X
patientIDs = gene_df.columns[2:].to_list()
# dead is 0, alive is 1
patientStatus = [0 if i <= 21 else 1 for i in range(1, 61)]

# Create the list of tuples
patients = list(zip(patientIDs, patientStatus))

#print(patients)

**Create Training and Testing Datasets.**


In [4]:
goodData = False # boolean to see if the data is good. About half of the dead 
                 # patients and half of the alive patients should be in the data 
                 # set of 30

attemptNumber = 0 # number of attempts to get good data

# empty lists that will contain all the patientIDs and their status for a training and testing dataset
patients_trainingSet = [None] * 30
patients_testingSet = [None] * 30


while not goodData:
    
    # add 1 to number of attempts to get good data
    attemptNumber +=1 
    
    # shuffle around patients
    random.shuffle(patients)
    
    # pick 30 random patients
    temp_trainingSet = patients[:30]
    
    # set number of dead patients in set variable to 0
    dead, alive = 0, 0

    # Count how many dead patients were selected for the set
    for patient in temp_trainingSet:
        if patient[1] == 0:
            dead+=1
        else:
            alive+=1
    
    # To stop infinite loops
    if attemptNumber == 10:
        break
    
    # If about half of the 39 alive patients are in this data set, populate the training and testing datasets, set goodData to True and break out of loop
    if 17 <= alive <= 22:
        print(alive, 'patients who responded and ', dead, 'patients who did not respond in testing dataset\n\n')
        patients_trainingSet = patients[:30]
        patients_testingSet = patients[30:]
        goodData = True
    else:
        # If attempt fails
        print("Failed attempt", attemptNumber)

# print training and datasets
#print('Training Dataset:\n', patients_trainingSet,'\n\n')
#print('Testing Dataset:\n', patients_testingSet)

18 patients who responded and  12 patients who did not respond in testing dataset




**Ensure data sets are good**
1. Ensure the correct number of patients are in the datasets
2. Ensure there is no overlap between data sets (no duplicate patients)

In [5]:
if len(patients_trainingSet) == 30:
    if len(patients_testingSet) == 30:
        print('The correct number of patients per dataset')
    else:
        print('panic', len(patients_trainingSet), len(patients_testingSet))

for train_patient in patients_trainingSet:
    for test_patient in patients_testingSet:
        if train_patient[0] == test_patient[0]:
            print('Panic, ', test_patiet[0], ' is duplicated')
            sys.exit()
    
print("No duplicate patients across training and testing datasets")

The correct number of patients per dataset
No duplicate patients across training and testing datasets


**K-Nearest Neighbor analysis**

**Craete a input**
1. outcome         -> a list of the outcomes for all the patients in the training data set
2. expressionLevel -> a list of lists. For every probe, each patient's expression level for that probe is consolidated into a list.

In [6]:
# Create a copy of the entire dataset
train = gene_df.copy()

# Get a list of patients that will be removed to create the training dataset
patTrain = [lisTrain[0] for lisTrain in patients_testingSet]

# for each patient that is in the testing set, drop it out of the training set
for patient in patTrain:
    train = train.drop(columns=[patient])

In [7]:
# A list of the outcomes (dead/alive) for all the patients in the training data set
trainOutcomes = [lis[1] for lis in patients_trainingSet]

# A list of lists. For every probe, each patient's expression level for that probe is consolidated into a list
train_expressionLevel = train.drop(train.iloc[:, 0:2], axis=1)

scaler = StandardScaler()
scaler.fit(train_expressionLevel)

#train_expressionLevel = train_expressionLevel.transpose()
train_expressionLevel = scaler.transform(train_expressionLevel).transpose()

**Function to find the best K-Nearest Neighbor classifier and parameters**

In [8]:
def knn_leave_one_out(X, y):
    loo = LeaveOneOut()
    
    # Define parameter grid for grid search
    param_grid = {'n_neighbors': range(1, 20)}  # This can be adjusted up to the number of patients in the dataset. Lower is better, too low is innacurate.
    
    # Initialize KNN classifier
    knn = KNeighborsClassifier()
    
    # Perform grid search with leave-one-out cross-validation
    grid_search = GridSearchCV(knn, param_grid, scoring='accuracy', cv=loo)
    grid_search.fit(X, y)
    
    # Get the best parameters
    best_params = grid_search.best_params_
    
    # Train the best KNN classifier on the entire dataset
    best_knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
    best_knn.fit(X, y)
    
    return best_knn, best_params

# Perform leave-one-out testing
best_knn, best_params = knn_leave_one_out(train_expressionLevel, trainOutcomes)
print("Best Parameters:", best_params)
print(best_knn)

Best Parameters: {'n_neighbors': 1}
KNeighborsClassifier(n_neighbors=1)


**Apply the classifier to the testing set and evaluate the accuracy**

In [9]:
# Create a copy of the entire dataset
test = gene_df.copy()

# Get a list of patients that will be removed to create the training dataset
patTest = [lisTest[0] for lisTest in patients_trainingSet]

# for each patient that is in the testing set, drop it out of the training set
for patient in patTest:
    test = test.drop(columns=[patient])

In [10]:
# A list of the outcomes (dead/alive) for all the patients in the training data set
testOutcomes = [lis[1] for lis in patients_trainingSet]

# A list of lists. For every probe, each patient's expression level for that probe is consolidated into a list
test_expressionLevel = test.drop(test.iloc[:, 0:2], axis=1)

scaler.fit(test_expressionLevel)

#train_expressionLevel = train_expressionLevel.transpose()
test_expressionLevel = scaler.transform(test_expressionLevel).transpose()

In [11]:
# Use the trained KNN classifier to make predictions on the test data
testPredictions = best_knn.predict(test_expressionLevel)

# Calculate the confusion matrix
confMatrix = confusion_matrix(testOutcomes, testPredictions)

# Fisher's exact test
oddsRatio, pValue = fisher_exact(confMatrix)
print("Fisher's exact test p-value:", pValue)

#Assuming you have predictions and true labels

mccPhi = matthews_corrcoef(testOutcomes, testPredictions)
print("Matthews' correlation coefficient Phi:", mccPhi)

Fisher's exact test p-value: 0.03415061699919271
Matthews' correlation coefficient Phi: 0.43082021842766455


In [12]:
predictions_0 = best_knn.predict(test_expressionLevel)

# Evaluate the performance using accuracy and other metrics
accuracy = accuracy_score(testOutcomes, predictions_0)

print("Accuracy on new data:", accuracy)

Accuracy on new data: 0.7333333333333333
